# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 12:11PM,257958,16,9191768,Sartorius Bioprocess Solutions,Released
1,Feb 22 2023 12:11PM,257957,10,37736,"Digital Brands, LLC",Released
2,Feb 22 2023 12:09PM,257956,19,TR-RM-02232023.1,"GCH Granules USA, Inc.",Released
3,Feb 22 2023 12:03PM,257955,16,9191746,Sartorius Bioprocess Solutions,Released
4,Feb 22 2023 12:02PM,257954,12,9191814,"SD Head USA, LLC",Released
5,Feb 22 2023 11:59AM,257953,16,SHL-9191810,"SHL Pharma, LLC",Released
6,Feb 22 2023 11:43AM,257952,10,9187452,Eywa Pharma Inc.,Released
7,Feb 22 2023 11:43AM,257952,10,9187478,Eywa Pharma Inc.,Released
8,Feb 22 2023 11:43AM,257952,10,9187525,Eywa Pharma Inc.,Released
9,Feb 22 2023 11:40AM,257947,10,PRF-53787,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,257954,Released,1
45,257955,Released,1
46,257956,Released,1
47,257957,Released,1
48,257958,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257954,NaN,NaN,1.0
257955,NaN,NaN,1.0
257956,NaN,NaN,1.0
257957,NaN,NaN,1.0
257958,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257845,0.0,0.0,2.0
257852,0.0,0.0,1.0
257858,0.0,1.0,0.0
257863,4.0,8.0,0.0
257864,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257845,0,0,2
257852,0,0,1
257858,0,1,0
257863,4,8,0
257864,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257845,0,0,2
1,257852,0,0,1
2,257858,0,1,0
3,257863,4,8,0
4,257864,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257845,,,2
1,257852,,,1
2,257858,,1,
3,257863,4,8,
4,257864,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions
1,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC"
2,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc."
3,Feb 22 2023 12:03PM,257955,16,Sartorius Bioprocess Solutions
4,Feb 22 2023 12:02PM,257954,12,"SD Head USA, LLC"
5,Feb 22 2023 11:59AM,257953,16,"SHL Pharma, LLC"
6,Feb 22 2023 11:43AM,257952,10,Eywa Pharma Inc.
9,Feb 22 2023 11:40AM,257947,10,Bio-PRF
10,Feb 22 2023 11:38AM,257951,19,"GCH Granules USA, Inc."
11,Feb 22 2023 11:38AM,257950,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,,,1
1,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",,,1
2,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",,,1
3,Feb 22 2023 12:03PM,257955,16,Sartorius Bioprocess Solutions,,,1
4,Feb 22 2023 12:02PM,257954,12,"SD Head USA, LLC",,,1
5,Feb 22 2023 11:59AM,257953,16,"SHL Pharma, LLC",,,1
6,Feb 22 2023 11:43AM,257952,10,Eywa Pharma Inc.,,,3
7,Feb 22 2023 11:40AM,257947,10,Bio-PRF,,,1
8,Feb 22 2023 11:38AM,257951,19,"GCH Granules USA, Inc.",,,1
9,Feb 22 2023 11:38AM,257950,19,"AdvaGen Pharma, Ltd",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,1,,
1,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",1,,
2,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",1,,
3,Feb 22 2023 12:03PM,257955,16,Sartorius Bioprocess Solutions,1,,
4,Feb 22 2023 12:02PM,257954,12,"SD Head USA, LLC",1,,
5,Feb 22 2023 11:59AM,257953,16,"SHL Pharma, LLC",1,,
6,Feb 22 2023 11:43AM,257952,10,Eywa Pharma Inc.,3,,
7,Feb 22 2023 11:40AM,257947,10,Bio-PRF,1,,
8,Feb 22 2023 11:38AM,257951,19,"GCH Granules USA, Inc.",1,,
9,Feb 22 2023 11:38AM,257950,19,"AdvaGen Pharma, Ltd",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,1,,
1,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",1,,
2,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",1,,
3,Feb 22 2023 12:03PM,257955,16,Sartorius Bioprocess Solutions,1,,
4,Feb 22 2023 12:02PM,257954,12,"SD Head USA, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",1.0,NaN,NaN
2,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Feb 22 2023 12:03PM,257955,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Feb 22 2023 12:02PM,257954,12,"SD Head USA, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 12:11PM,257958,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Feb 22 2023 12:11PM,257957,10,"Digital Brands, LLC",1.0,0.0,0.0
2,Feb 22 2023 12:09PM,257956,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Feb 22 2023 12:03PM,257955,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Feb 22 2023 12:02PM,257954,12,"SD Head USA, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5931823,81.0,14.0,12.0
12,1031660,7.0,2.0,0.0
15,257890,38.0,31.0,0.0
16,773866,3.0,0.0,0.0
18,257920,1.0,0.0,0.0
19,2063347,9.0,4.0,0.0
21,515891,2.0,0.0,0.0
22,773663,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5931823,81.0,14.0,12.0
1,12,1031660,7.0,2.0,0.0
2,15,257890,38.0,31.0,0.0
3,16,773866,3.0,0.0,0.0
4,18,257920,1.0,0.0,0.0
5,19,2063347,9.0,4.0,0.0
6,21,515891,2.0,0.0,0.0
7,22,773663,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,81.0,14.0,12.0
1,12,7.0,2.0,0.0
2,15,38.0,31.0,0.0
3,16,3.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,9.0,4.0,0.0
6,21,2.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,81.0
1,12,Released,7.0
2,15,Released,38.0
3,16,Released,3.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Completed,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,14.0,2.0,31.0,0.0,0.0,4.0,0.0,0.0
Released,81.0,7.0,38.0,3.0,1.0,9.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,2.0,31.0,0.0,0.0,4.0,0.0,0.0
2,Released,81.0,7.0,38.0,3.0,1.0,9.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,2.0,31.0,0.0,0.0,4.0,0.0,0.0
2,Released,81.0,7.0,38.0,3.0,1.0,9.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()